In [ ]:
import utils
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.set(rc = {'figure.figsize':(42,30)}, font_scale = 2)

In [ ]:
engine = utils.get_engine()

In [ ]:
def query (symb) :
    return f"""
            drop table if exists analytics.{symb}_monthly;
            create table analytics.{symb}_monthly as (
with numbered_rows as (
select s.date,
       date_trunc('month', s.date) as month,
       row_number() over (partition by to_char(s.date, 'yyyy'), date_trunc('month', s.date) order by s.date)=1 as is_first_row,
       row_number() over (partition by to_char(s.date, 'yyyy'), date_trunc('month', s.date) order by s.date desc)=1 as is_last_row,
       s.volume as stock_volume,
       s.open as stock_open,
       s.high as stock_high,
       s.low as stock_low,
       s.close as stock_close
from stocks.{symb}_1d s
where date>='2000-01-01'
)
    select month,
           sum(stock_volume) as volume,
           max(case when is_first_row then stock_open else -1 end) as open,
           max(case when is_last_row then stock_close else -1 end) as close,
           (max(case when is_last_row then stock_close else -1 end)/max(case when is_first_row then stock_open else -1 end) - 1) as return,
           min(stock_low) as low,
           max(stock_high) as high,
           max(stock_volume) max_volume,
           min(stock_volume) min_volume
    from numbered_rows
    group by 1)
            """

def return_query(symbols_list):
    query_select = """select  spy.month,"""
    query_join = """from analytics.spy_monthly spy """
    
    for symbol in symbols_list[:-1]:
        query_select+=f"\n {symbol}.close/lag({symbol}.close,1)over(order by spy.month) - 1 as {symbol}_return,"
        query_join+=f"""\n inner join analytics.{symbol}_monthly as  {symbol} on {symbol}.month::date=spy.month::date"""
    
    query_select+=f"\n {symbols_list[-1]}.close/lag({symbols_list[-1]}.close,1)over(order by spy.month) - 1 as {symbols_list[-1]}_return"
    query_join+=f"""\n inner join analytics.{symbols_list[-1]}_monthly as  {symbols_list[-1]} on {symbols_list[-1]}.month::date=spy.month::date"""
    query_join+="""\n order by 1;"""
    
    return query_select+"\n"+query_join



In [ ]:
def rolling_sharpe_ratio(returns, risk_free_return, periods):
    # As risk free return comes in annual yields, we should convert it to monthly return.
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    if len(returns.shape) == 1:
        excess_return=returns-risk_free_return
    else:
        excess_return = returns.apply(lambda x: x-risk_free_return)
    sr = excess_return.rolling(periods,min_periods=6).mean()/excess_return.rolling(periods,min_periods=6).std()
    return sr

def calculate_sharpe_ratio(returns, risk_free_return):
    # As risk free return comes in annual yields, we should convert it to monthly return.
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    excess_return = returns.apply(lambda x: x-risk_free_return)
    return excess_return.mean(axis=0)/excess_return.std(ddof=1,axis=0)

In [ ]:
def rolling_sortino_ratio(returns, risk_free_return, periods):
    # As risk free return comes in annual yields, we should convert it to monthly return.
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    if len(returns.shape) == 1:
        excess_return=returns-risk_free_return
    else:
        excess_return = returns.apply(lambda x: x-risk_free_return)
    sortino = excess_return.rolling(periods,min_periods=6).mean()/excess_return.where(excess_return<0, 0.01).rolling(periods,min_perdiods=6).std()
    return sortino

def calculate_sortino_ratio(returns, risk_free_return):
    # As risk free return comes in annual yields, we should convert it to monthly return.
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    if len(returns.shape) == 1:
        excess_return=returns-risk_free_return
    else:
        excess_return = returns.apply(lambda x: x-risk_free_return)
    return excess_return.mean(axis=0)/excess_return.where(excess_return<0, 0.01).std(axis=0)

In [ ]:
def rolling_calmar_ratio(returns, risk_free_return, periods=36):
    #annualized_returns = returns.apply(lambda x: np.log(x+1)).rolling(periods).mean().apply(lambda x: (np.exp(x)**12.0)-1)
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    if len(returns.shape) == 1:
        excess_return=returns-risk_free_return
    else:
        excess_return = returns.apply(lambda x: x-risk_free_return)
    
    calmar = excess_return.rolling(periods,min_periods=6).mean()/excess_return.where(excess_return<0, 0.01).rolling(periods,min_periods=6).min().abs()
    return calmar

def calculate_calmar_ratio(returns, risk_free_return, periods=36):
    #annualized_returns = returns.apply(lambda x: np.log(x+1)).rolling(periods).mean().apply(lambda x: (np.exp(x)**12.0)-1)
    risk_free_return = risk_free_return.apply(lambda x: ((x+1)**(1/12))-1)
    if len(returns.shape) == 1:
        excess_return=returns-risk_free_return
    else:
        excess_return = returns.apply(lambda x: x-risk_free_return)
        
    return excess_return.tail(periods).mean(axis=0)/excess_return.tail(periods).where(excess_return<0, 0.01).min(axis=0).abs()



In [ ]:
def draw_hist_and_dist(data):
    sns.set(rc = {'figure.figsize':(42,30)}, font_scale = 2)
    mu, std = norm.fit(data) 
    plt.hist(data, bins='auto', density=True, alpha=0.6, color='b')
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 1000)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=3)
    title = "Mean={:.2f} / Std={:.2f}".format(mu, std)
    plt.title(title)
    plt.show()

In [ ]:
symbols = ['spy', 'qqq', 'dia', 'eem', 'efa', 'ewj',
           'ivw', 'vti', 'xle', 'xlf', 'xlk', 'psj', 'itot', 
           'btcusd', 'ethusd', 'xmrusd', 'socl', 'haus', 
           'arkk', 'arkw', 'arkf', 'arkq',
           'awk', 'pho', 'pio', 'cgw',
           'aapl','msft','goog','amzn','tsla','nvda','meta','xom', 'dow',
           'unh','jnj','wmt','cvx','hd','abbv','ko','baba','pfe','mcd',
           'abt','vz','amd','sbux','ibm','spgi','intc','blk','amat',
           'adp','pypl','meli','pnc','cl','sq','ndaq','awk','spot','xyl','awr',
           'treasury_yield_13_weeks',
           'treasury_yield_5_years', 'treasury_yield_10_years']

In [ ]:
with engine.connect() as connection:
    for s in symbols:
        try:
            connection.execute(query(s))
            print(f"Monthly market data created for symbol {s}.")
        except:
            print(f"Error with symbol {s}")
    connection.close()

In [ ]:
selected_constituents = list(set(['qqq', 'dia', 'eem', 'efa', 'ewj', 'vti', 
                                 'xle', 'xlf', 'xlk', 'psj', 'itot', 'awk', 'pho', 'pio', 'cgw',
                               'aapl','msft','goog','amzn','tsla','nvda','meta','xom',
                               'unh','jnj','wmt','cvx','hd','abbv','ko','baba','pfe','mcd',
                               'abt','vz','amd','sbux','ibm','spgi','intc','blk','amat',
                               'adp','pypl','meli','pnc','cl','sq','ndaq','awk','xyl','awr', 'treasury_yield_13_weeks']))

In [ ]:
with engine.connect() as connection:
    query = return_query(selected_constituents)
    df_return = pd.read_sql(query, con=connection).set_index('month').dropna()
    connection.close()

In [ ]:
annualization_factor = 12**0.5

In [ ]:
df_return.info()

In [ ]:
selected_equities = list(set(df_return.columns.tolist())-set(['treasury_yield_13_weeks_return']))


In [ ]:
excess_return.rolling(periods).min().abs()